In [ ]:
import cv2
import numpy as np

In [ ]:
ESC=0
img1 = cv2.imread('beer2.png')
gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
orb = cv2.ORB_create()

In [ ]:
cap = cv2.VideoCapture('beer_table.avi') 

while(cap.isOpened()):  
    (grabbed, frame) = cap.read()  
    img2 = frame.copy()

    cv2.namedWindow('frame')  
    gray2 = cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY) 
    
    kpts1, descs1 = orb.detectAndCompute(gray1,None)
    kpts2, descs2 = orb.detectAndCompute(gray2,None)

    ## match descriptors and sort them in the order of their distance
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = bf.match(descs1, descs2)
    dmatches = sorted(matches, key = lambda x:x.distance)

    ## extract the matched keypoints
    src_pts  = np.float32([kpts1[m.queryIdx].pt for m in dmatches]).reshape(-1,1,2)
    dst_pts  = np.float32([kpts2[m.trainIdx].pt for m in dmatches]).reshape(-1,1,2)

    ## find homography matrix and do perspective transform
    M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,5.0)
    h,w = img1.shape[:2]
    pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)
    dst = cv2.perspectiveTransform(pts,M)

    ## draw found regions
    img2 = cv2.polylines(img2, [np.int32(dst)], True, (0,0,255), 1, cv2.LINE_AA)
    cv2.imshow("found", img2)

    ## draw match lines
    res = cv2.drawMatches(img1, kpts1, img2, kpts2, dmatches[:20],None,flags=2)

    cv2.imshow("orb_match", res);
    
    key = cv2.waitKey(0)                                 
    if key == ESC:
        break

    
    
cap.release()  
cv2.destroyAllWindows()